In [1]:
import psycopg2 as ps
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem as ch

## Load the DrugBank SDF

In [2]:
suppl = ch.SDMolSupplier('/home/uzivatel/data/ivan/projekty/chemspacescripts/scratch/drugbank.sdf')
mols = [x for x in suppl if x is not None]

## Connect to the database

In [20]:
connection = ps.connect(user='uzivatel', dbname='drugbank', port='9000') # change as needed
cursor = connection.cursor()

## Insert the molecules

In [4]:
cursor.execute("delete from drugbank.drugbank") # in case there are old values
query = "insert into drugbank.drugbank (smiles, id) values %s" %((', '.join(["('%s', '%s')" % (ch.MolToSmiles(mol), mol.GetProp('DRUGBANK_ID'))
                                                                         for mol in mols])))
cursor.execute(query)
cursor.execute("select * from drugbank.drugbank limit 5")
print(cursor.fetchall())

[('DB00006', 'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O', None, None, None, None, None, None), ('DB00014', 'CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1cnc[nH]1)NC(=O)C1CCC(=O)N1)C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NNC(N)=O', None, None, None, None, None, None), ('DB00035', 'N=C(N)NCCCC(NC(=O)C1CCCN1C(=O)C1CSSCCC(=O)NC(Cc2ccc(O)cc2)C(=O)NC(Cc2ccccc2)C(=O)NC(CCC(N)=O)C(=O)NC(CC(N)=O)C(=O)N1)C(=O)NCC(N)=O', None, None, None, None, None, None), ('DB00050', 'CC(=O)NC(Cc1ccc2ccccc2c1)C(=O)NC(Cc1ccc(Cl)cc1)C(=O)NC(Cc1cccnc1)C(=O)NC(CO)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CCCNC(N)=O)C(=O)NC(CC(C)C)C(=O)NC(CCCNC(=N)N)C(=O)N1CCCC1C(=O)NC(C)C(N)=O', None, None, None, None, None, None), ('DB00091', 'CC=CCC(C)C(O)C1C(=O)N

## Create the mol instances

In [5]:
cursor.execute("update drugbank.drugbank set molinstance = mol_from_smiles(smiles::cstring)")
cursor.execute("select * from drugbank.drugbank limit 5")
print(cursor.fetchall())

[('DB00006', 'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O', 'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCNC(=N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O', None, None, None, None, None), ('DB00116', 'Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1)N2', 'Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1)N2', None, None, None, None, None), ('DB00117', 'NC(Cc1cnc[nH]1)C(=O)O', 'NC(Cc1cnc[nH]1)C(=O)O', None, None, None, None, None), ('DB00118', 'C[S+](CCC(N)C(=O)O)CC1OC(n2cnc3c(N)ncnc32)C(O)C1O', 'C[S+](CCC(N)C(=O)O)CC1OC(n2cnc3c(N)ncnc32)C(O)C1O', None, No

## Calculate the descriptors

In [6]:
cursor.execute("update drugbank.drugbank set (hba, hbd, mw, logp) = (mol_hba(molinstance), mol_hbd(molinstance), mol_amw(molinstance), mol_logp(molinstance))")
cursor.execute("select * from drugbank.drugbank limit 5")
print(cursor.fetchall())

[('DB09479', '[Cl-].[Rb+]', '[Cl-].[Rb+]', None, 0, 0, 120.920997619629, -5.99200010299683), ('DB03174', 'O=CCP(=O)(O)O', 'O=CCP(=O)(O)O', None, 4, 2, 124.031997680664, -0.637000024318695), ('DB03175', 'CCCO', 'CCCO', None, 1, 1, 60.0960006713867, 0.388700008392334), ('DB01370', '[Al]', '[Al]', None, 0, 0, 26.9820003509521, -0.380800008773804), ('DB01440', 'O=C([O-])CCCO', 'O=C([O-])CCCO', None, 3, 1, 103.09700012207, -1.49119997024536)]


## Calculate Morgan fingerprints

In [7]:
cursor.execute("update drugbank.drugbank set morganfp = morganbv_fp(molinstance)")
cursor.execute("select * from drugbank.drugbank limit 5")
print(cursor.fetchall())

[('DB02679', 'N#CN', 'N#CN', '\\x00000000000000000000000000000000000008000440000000000000000000000000400000000000000000000001080000000000000000000000000000000000', 2, 2, 42.0410003662109, -0.573819994926453), ('DB02688', 'C=C(N)C(=O)O', 'C=C(N)C(=O)O', '\\x00000000000000000000000000000000001408000000000000000000000000000000100080000000008040000000002020000000000000400000000000000040', 3, 3, 87.0780029296875, -0.45660001039505), ('DB03085', 'O=C(O)CO', 'O=C(O)CO', '\\x00000000000000000000010000000000010400000000000000020040000000000000000080000000000000000000012020000000000000000000000000000000', 3, 2, 76.0510025024414, -0.936699986457825), ('DB03226', 'OCC(F)(F)F', 'OCC(F)(F)F', '\\x00000000000000000000010000000c00000000000000000000002040000000000000000080000000000000000000000000010000002000000000000000000000', 1, 1, 100.039001464844, 0.541000008583069), ('DB03399', 'C#[N+]CC', 'C#[N+]CC', '\\x000000000200040000000100000000000000000000000000000000000000000000000000400000000000000000000

## Find and insert the drug groups into the database

In [8]:
grouplists = (mol.GetProp('DRUG_GROUPS').split('; ') for mol in mols)
groupnames = set()
for grouplist in grouplists:
    groupnames.update(grouplist)
cursor.execute("delete from drugbank.druggroup") # in case there are old values
query = "insert into drugbank.druggroup (groupname) values %s" % (', '.join(["('%s')" % (groupname)
                                                                             for groupname in groupnames]))
cursor.execute(query)

# let's get the database-assigned ID's
cursor.execute("select * from drugbank.druggroup")
ids_groupnames = cursor.fetchall()
print(ids_groupnames)
# and make a convenient dict out of them
groupname2id = { groupname:gid for gid, groupname in ids_groupnames}
print(groupname2id)

[(1, 'vet_approved'), (2, 'approved'), (3, 'nutraceutical'), (4, 'illicit'), (5, 'investigational'), (6, 'withdrawn'), (7, 'experimental')]
{'vet_approved': 1, 'approved': 2, 'nutraceutical': 3, 'illicit': 4, 'investigational': 5, 'withdrawn': 6, 'experimental': 7}


## Create the drug <--> druggroup connection table

In [9]:
drugbank_id__druggroup_id = []
for mol in mols:
    molid = mol.GetProp('DRUGBANK_ID')
    drugbank_id__druggroup_id.extend([(molid, groupname2id[groupname]) for groupname
                                        in mol.GetProp('DRUG_GROUPS').split('; ')])
cursor.execute("delete from drugbank.druggroup2drugbank") # in case there are old values
query = "insert into drugbank.druggroup2drugbank (drugbank_id, druggroup_id) values %s" % ', '.join(["('%s', '%s')" % (drugbank_id, druggroup_id)
                                                                for drugbank_id, druggroup_id in drugbank_id__druggroup_id])
cursor.execute(query)
cursor.execute("select * from drugbank.druggroup2drugbank limit 5")
print(cursor.fetchall())
connection.commit() # just to make sure

[('DB00006', 2), ('DB00006', 5), ('DB00014', 2), ('DB00035', 2), ('DB00050', 2)]


## Get illicit drugs

In [10]:
cursor.execute("""select * from drugbank.druggroup g 
                    left join drugbank.druggroup2drugbank gd on (g.id = gd.druggroup_id)
                    left join drugbank.drugbank d on (gd.drugbank_id = d.id) where groupname = 'illicit'""")
result = cursor.fetchall()
print(len(result), result[:5])

(186, [(4, 'illicit', 'DB00182', 4, 'DB00182', 'CC(N)Cc1ccccc1', 'CC(N)Cc1ccccc1', '\\x02800000020480000180010000000000000008000080000000004004000000000000040800000000000002021004000020000000000000000000000000000000', 1, 2, 135.210006713867, 1.57630002498627), (4, 'illicit', 'DB00189', 4, 'DB00189', 'C#CC(O)(C=CCl)CC', 'C#CC(O)(C=CCl)CC', '\\x400000001200000100000100002004000000080004004000000000000000000000000000e0000040000000000000180000000800000000008040000004000080', 1, 1, 144.600997924805, 1.51320004463196), (4, 'illicit', 'DB00191', 4, 'DB00191', 'CC(C)(N)Cc1ccccc1', 'CC(C)(N)Cc1ccccc1', '\\x00000000024400000100010000000400800008400980000000004004000000000000000000000000000002001000000120010000000000000000000000000000', 1, 2, 149.236999511719, 1.96640002727509), (4, 'illicit', 'DB00230', 4, 'DB00230', 'CC(C)CC(CN)CC(=O)O', 'CC(C)CC(CN)CC(=O)O', '\\x020000000a0000000000010000002000000408400001000800020000000080000004000880008000000000000000202020100000000000000004200000000000', 3,

## Find and insert the product names

In [21]:
product_name__drugbank_id = []
for mol in mols:
    molid = mol.GetProp('DRUGBANK_ID')
    try:
        product_name__drugbank_id.extend([(productname.replace("'", ""), molid) for productname
                                            in mol.GetProp('PRODUCTS').split('; ')])
    except KeyError:
        pass
print(len(product_name__drugbank_id), product_name__drugbank_id[:5])
cursor.execute("delete from drugbank.products") # in case there are old values
query = "insert into drugbank.products (productname, drugbank_id) values %s" % ', '.join(["('%s', '%s')" % (productname, drugbank_id)
                                                                for product_name, drugbank_id in product_name__drugbank_id])
cursor.execute(query)
cursor.execute("select * from drugbank.products limit 5")
print(cursor.fetchall())
connection.commit() # just to make sure

(78076, [('Angiomax', 'DB00006'), ('Bivalirudin', 'DB00006'), ('Bivalirudin for Injection', 'DB00006'), ('Zoladex', 'DB00014'), ('Zoladex Inj Depot 3.6mg', 'DB00014')])
[('Spectro Eczemacare Medicated Cream', 'DB00006'), ('Spectro Eczemacare Medicated Cream', 'DB00006'), ('Spectro Eczemacare Medicated Cream', 'DB00006'), ('Spectro Eczemacare Medicated Cream', 'DB00014'), ('Spectro Eczemacare Medicated Cream', 'DB00014')]


## Close the connections

In [19]:
cursor.close()
connection.close()